# Assignment 2 - Solution
---

### Step 3: Strategy Selection

- The following strategies are selected :
    + SMA
    + EMA
    + RSI
    + Donchannel
    + Bollinger Bands

### Step 4 : Backtesting

The backtesting is performed on all the data for ICICI Bank & Kotak NIFTY50 Value 20 ETF using two methods
- The first method is using the pyfolio library, which is a external library 
- Custom backtesting using all the formulas shared in the call 


> There is a difference in results of backtesting with pyfolio and custom formulas shared in class due to :
> 1. The pyfolio assumes the value 0 for 'risk free returns, for custom calculations I am taking the value 0.067188
> 2. The number of days I am assuming is 252, but pyfolio using different methods
> 3. The above two calculations effect annual returns calculations, which in-term effect other calculations ( e.g. sharpe ratio )
>
> Due to these reason, I've done calculations with both methods


### Step 5: Performance Metrics

- The following metrics are calculated in this assignment:
    + Custom Ratios
        - Annual Volatility 
        - Sharpe Ratio
        - Max Drawdown
        - Calmer Ratio
        - Daily Value @ Risk
        - Sortino Ratio
        - Tail Ratio

Environment Setting **Must be followed for execution**
1. This ==program does not work with python 3.12==, it only works with 3.11
2. The pyfolio library is not actively maintained currently, therefore only ==pandas version 1.5.7 works. Any version of Pandas > 2.0.0 is not supported==

In [ ]:
'''
This block only installs the necessary dependencies onto the system. If the libs are already installed, then no need to execute this command.

Known issues 
- The pyfolio lib only works with pandas version < 2.0.0, therefore the highest version below 2.0.0 ( i.e. 1.5.3 ) is used

'''
%pip install pandas==1.5.3
%pip install yfinance==0.2.37
%pip install pyfolio=0.9.2
%pip install empyrical=0.5.5
%pip install pydantic==2.6.3
%pip install matplotlib==3.8.3
%pip install ta==0.11.0


In [ ]:
# import the libs
import pandas as pd
import pandas as pd
import yfinance as yf
import pyfolio as pf
import datetime as dt
import empyrical as em
import ta
import numpy as np
from pydantic import BaseModel
import warnings
from scipy.stats import norm

# Ignore printing all warnings
warnings.filterwarnings('ignore')

In [ ]:
class Settings( BaseModel ):
    period   : str = "1y"
    interval : str = "1d"
    tmp_folder: str = 'tmp'
    yahoo_ticker_ext: str = ".NS"
    risk_free_rate: float = 0.067188
    days_in_year: int = 252

setting = Settings( )

# dataframes to hold the metrics related data
descriptive_stats = pd.DataFrame( ) 
custom_metrics = pd.DataFrame( )
pyfolio_metrics = pd.DataFrame( )

def get_performance_metrics( return_data ):

    # calcuate the annual ration
    annual_return       = np.mean( return_data ) * setting.days_in_year
    
    # calculate annual volatility ratio
    annual_volatility   = np.std(return_data) * np.sqrt( setting.days_in_year )
    
    # calculate sharpe ratio
    sharpe_ratio        = (annual_return - setting.risk_free_rate) / annual_volatility

    # calculate rolling max window
    rolling_max = return_data.cummax()

    # daily drawdown
    daily_drawdown = return_data / rolling_max - 1.0
    
    max_drawdown = daily_drawdown.cummin().min()

    calmar_ratio = (annual_return - setting.risk_free_rate ) / abs(max_drawdown)

    daily_var = norm.ppf(1-0.05, np.mean(return_data), np.std(return_data))

    negative_std = np.std( return_data [return_data < 0])
    sortino_ratio = (annual_return -setting.risk_free_rate)/(negative_std * np.sqrt(252))

    positive_returns_sum = np.sum( return_data[return_data > 0])
    negative_returns_sum = np.sum( return_data[return_data < 0])
    omega_ratio = positive_returns_sum / abs(negative_returns_sum)

    tail_ratio = abs(np.percentile(return_data.dropna(), 95)) / abs(np.percentile( return_data.dropna(), 5))

    metrics = {
        'annual_return': annual_return,
        'annual_volatility': annual_volatility,
        'sharpe_ratio' : sharpe_ratio,
        'max_drawdown': max_drawdown,
        'calmar_ratio': calmar_ratio,
        'daily_value_at_risk': daily_var,
        'sortino_ratio': sortino_ratio,
        'omega_ratio': omega_ratio,
        'tail_ratio': tail_ratio
    }

    return metrics

# download the data from yfinance
stock_symbol = "ICICIBANK.NS" 
drop_column_list =  ['Open', 'Volume', 'Stock Splits', 'Dividends']

ticker = yf.Ticker( stock_symbol )
ticker_df = ticker.history( period=setting.period, interval=setting.interval )
ticker_df.dropna(inplace = True)
ticker_df['daily_returns'] = np.log(ticker_df['Close']/ticker_df['Close'].shift(1))

ticker_df.drop( drop_column_list ,axis=1, inplace=True, errors='ignore')

ticker_df.head( )

In [ ]:
# Setting up the calculations for Simple Moving Average - Cross Over

df_sma = ticker_df.copy( deep=True )
long_window  = 20
short_window = 10

df_sma[ 'short'] = ta.trend.SMAIndicator( df_sma['Close'], window=short_window).sma_indicator()
df_sma[ 'long' ] = ta.trend.SMAIndicator( df_sma['Close'], window=long_window ).sma_indicator()

# SMA Cross over Strategy 

# Position Strategy
# BUY condition
df_sma['sma_signal'] = np.where((df_sma['short'] > df_sma['long']),1,np.nan)

# SELL condition
df_sma['sma_signal'] = np.where( (df_sma['short'] < df_sma['long']),0,df_sma['sma_signal'])

# creating long and short positions
df_sma['sma_position'] = df_sma['sma_signal'].replace(to_replace=np.nan, method='ffill')

# shifting by 1, to account of close price return calculations
df_sma['sma_position'] = df_sma['sma_position'].shift(1)

# calculating returns
df_sma['returns'] = df_sma['daily_returns'] * (df_sma['sma_position'])

# add the descriptive statistics to the list
descriptive_stats['sma'] = df_sma['returns'].describe()

# add the custom metrics to the dataframe for comparison
tmp = get_performance_metrics( df_sma['returns'].dropna( ))
custom_metrics['sma'] = pd.Series( tmp.values(), index=tmp.keys())

# add the pyfolio to the dataframe for comparison
pf.create_simple_tear_sheet( df_sma['returns'] )
pyfolio_metrics['sma'] = pf.timeseries.perf_stats(  df_sma['returns'].dropna( ) )


In [ ]:
# Setting up the calculations for RSI 

df_rsi = ticker_df.copy(deep=True)
rsi_window = 14

rsi_upper_limit = 80
rsi_lower_limit = 29

df_rsi['RSI'] = ta.momentum.RSIIndicator( df_rsi['Close'], window = rsi_window ).rsi()

#generate signal
df_rsi['rsi_signal'] = 1 #initialize with default value

# BUY condition
df_rsi['rsi_signal'] = np.where((df_rsi['RSI'] > rsi_upper_limit),0,np.nan)

# SELL condition
df_rsi['rsi_signal'] = np.where( (df_rsi['RSI'] < rsi_lower_limit ),1,df_rsi['rsi_signal'])

df_rsi['rsi_signal_shift' ] = df_rsi['rsi_signal'].shift(1)

# update the signal keep the buy or sell signal is in the middle of upper & lower limit

df_rsi['rsi_signal'] = np.where ( ( df_rsi['RSI'] < rsi_upper_limit ) & ( df_rsi['RSI'] > rsi_lower_limit ), df_rsi['rsi_signal_shift'], df_rsi['rsi_signal'] )

#generate positions
# creating long and short positions
df_rsi['rsi_position'] = df_rsi['rsi_signal'].replace(to_replace=np.nan, method='ffill')

#calculate returns
df_rsi['daily_returns'] = np.log(df_rsi['Close'] / df_rsi['Close'].shift(1))

df_rsi['returns'] = df_rsi['daily_returns'] * (df_rsi['rsi_position'])

# add the descriptive statistics to the list
descriptive_stats['rsi'] = df_rsi['returns'].describe()

# add the custom metrics to the dataframe for comparison
tmp = get_performance_metrics( df_rsi['returns'].dropna( ))
custom_metrics['rsi'] = pd.Series( tmp.values(), index=tmp.keys())

# add the pyfolio to the dataframe for comparison
pf.create_simple_tear_sheet( df_rsi['returns'] )
pyfolio_metrics['rsi'] = pf.timeseries.perf_stats(  df_rsi['returns'].dropna( ) )




In [ ]:
# Setting up the calculations for Exponential Moving Average - Cross Over

df_ema = ticker_df.copy( deep=True )
long_window  = 20
short_window = 10

df_ema[ 'short'] = ta.trend.EMAIndicator( df_sma['Close'], window=short_window).ema_indicator()
df_ema[ 'long' ] = ta.trend.EMAIndicator( df_sma['Close'], window=long_window ).ema_indicator()

# EMA Cross over Strategy 

# Position Strategy
# BUY condition
df_ema['ema_signal'] = np.where((df_ema['short'] > df_ema['long']),1,np.nan)
# SELL condition
df_ema['ema_signal'] = np.where( (df_ema['short'] < df_ema['long']),0,df_ema['ema_signal'])

# creating long and short positions
df_ema['ema_position'] = df_ema['ema_signal'].replace(to_replace=np.nan, method='ffill')

# shifting by 1, to account of close price return calculations
df_ema['ema_position'] = df_ema['ema_position'].shift(1)

# calculating returns
df_ema['returns'] = df_ema['daily_returns'] * (df_ema['ema_position'])

# add the descriptive statistics to the list
descriptive_stats['ema'] = df_ema['returns'].describe()

# add the custom metrics to the dataframe for comparison
tmp = get_performance_metrics( df_ema['returns'].dropna( ))
custom_metrics['ema'] = pd.Series( tmp.values(), index=tmp.keys())

# add the pyfolio to the dataframe for comparison
pf.create_simple_tear_sheet( df_ema['returns'] )
pyfolio_metrics['ema'] = pf.timeseries.perf_stats(  df_ema['returns'].dropna( ) )


In [ ]:
# Setting up Donchain Channel 

df_dcc = ticker_df.copy( deep=True )
#df_dcc.drop( 'daily_returns', inplace=True)

# Define the Donchian Channel window size
channel_window = 5

# Calculate the Donchian Channels
high = df_dcc['High'].rolling(window=channel_window).max()
low = df_dcc['Low'].rolling(window=channel_window).min()

upper_channel = high.shift(1)
lower_channel = low.shift(1)

# Generate trading signals
df_dcc['Signal'] = np.where(df_dcc['Close'] > upper_channel, 1, np.nan)
df_dcc['Signal']= np.where(df_dcc['Close'] < lower_channel, 0, df_dcc['Signal'])
df_dcc['position'] = df_dcc['Signal'].replace(to_replace=np.nan, method='ffill')

# Calculate daily returns
df_dcc['daily_returns'] = np.log(df_dcc['Close'] / df_dcc['Close'].shift(1))
df_dcc['returns'] = np.log(df_dcc['Close'] / df_dcc['Close'].shift(1)) * df_dcc['position'].shift(1)

# add the descriptive statistics to the list
descriptive_stats['dcc'] = df_dcc['returns'].describe()

# add the custom metrics to the dataframe for comparison
tmp = get_performance_metrics( df_dcc['returns'].dropna( ))
custom_metrics['dcc'] = pd.Series( tmp.values(), index=tmp.keys())

# add the pyfolio to the dataframe for comparison
pf.create_simple_tear_sheet( df_dcc['returns'] )
pyfolio_metrics['dcc'] = pf.timeseries.perf_stats(  df_dcc['returns'].dropna( ) )

In [ ]:
#Bollinger Band Calculation

df_bb = ticker_df.copy( deep=True )

# Define the Bollinger Band Parameters
n = 10 # period for moving averages
multiplier_L = 1
multiplier_R = 1.25

# Calculate the Bollinger Bands
bollinger_mean = df_bb['Close'].rolling(window=n).mean()
bollinger_sd = df_bb['Close'].rolling(window=n).std()

df_bb['bb_high'] = bollinger_mean + multiplier_R * bollinger_sd
df_bb['bb_low' ] = bollinger_mean - multiplier_L * bollinger_sd

# Generate trading signals breakout str
df_bb['Signal'] = np.where(df_bb['Close'] >df_bb['bb_high'], 1, np.nan)
df_bb['Signal']= np.where(df_bb['Close'] < df_bb['bb_low'], 0, df_bb['Signal'])
df_bb['position'] = df_bb['Signal'].replace(to_replace=np.nan, method='ffill')

df_bb['returns'] = np.log(df_bb['Close'] / df_bb['Close'].shift(1)) * df_bb['position'].shift(1)

# add the descriptive statistics to the list
descriptive_stats['bb'] = df_bb['returns'].describe()

# add the custom metrics to the dataframe for comparison
tmp = get_performance_metrics( df_bb['returns'].dropna( ))
custom_metrics['bb'] = pd.Series( tmp.values(), index=tmp.keys())

# add the pyfolio to the dataframe for comparison
pf.create_simple_tear_sheet( df_dcc['returns'] )
pyfolio_metrics['bb'] = pf.timeseries.perf_stats(  df_bb['returns'].dropna( ) )



In [ ]:
print( "Descriptive Stats" )
descriptive_stats.head(20)

In [ ]:
print( "Custom Metrics")
custom_metrics.head( 20 )

In [ ]:
print ( "PyFolio Metrics")
pyfolio_metrics.head(20)